# Pockitect Colab Training (Unsloth)

This notebook fine-tunes the model on a GPU-backed Colab runtime using the project training data.

**Before you start:** Runtime → Change runtime type → **GPU**

## 1) Clone the repo

In [ ]:
%cd /content
!rm -rf pockitect-mvp
!git clone https://github.com/BrendenKennedy/pockitect-mvp.git
%cd /content/pockitect-mvp

Cloning into 'pockitect-mvp'...
remote: Enumerating objects: 477, done.
remote: Counting objects: 100% (477/477), done.
remote: Compressing objects: 100% (278/278), done.
remote: Total 477 (delta 193), reused 476 (delta 192), pack-reused 0 (from 0)
Receiving objects: 100% (477/477), 501.88 KiB | 2.61 MiB/s, done.
Resolving deltas: 100% (193/193), done.
/content/pockitect-mvp/pockitect-mvp/pockitect-mvp


## 2) Install dependencies

In [11]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install -r requirements.txt

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-p3gtg8j3/unsloth_3b56b28ee1594dd0be8a88b37e308279
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-p3gtg8j3/unsloth_3b56b28ee1594dd0be8a88b37e308279
  Resolved https://github.com/unslothai/unsloth.git to commit ab4061e106792fa91e1eba3e4f3d45fa8aba121e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


## 3) (Optional) Validate training data

In [12]:
!python tools/validate_training_data.py

🔍 Validating 162 training examples...


Validation Summary
Total files:     162
Valid:           162 ✓
Invalid:         0 ✗
Warnings:        0 ⚠️
Errors:          0

✅ All training files are valid!


## 4) (Optional) Regenerate JSONL if you changed training data

In [13]:
!python tools/convert_to_finetuning_format.py --format unsloth

📚 Loaded 162 training pairs

✅ Saved 162 examples to data/finetuning/train_unsloth.jsonl

✅ Conversion complete! Format: unsloth
📁 Output: data/finetuning/train_unsloth.jsonl


## 5) Train

In [ ]:
!python tools/finetune_model.py \
    --model unsloth/Qwen2.5-3B \
    --train data/finetuning/train_unsloth.jsonl \
    --epochs 3 \
    --batch-size 2 \
    --early-stopping \
    --early-stopping-patience 3 \
    --eval-steps 50

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
2026-01-19 18:25:44.233387: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768847144.253910    3026 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768847144.260007    3026 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768847144.275083    3026 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768847144.275106    3026 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:17

## 6) (Optional) Create Ollama model from output

Only if you plan to use Ollama locally after training.

In [ ]:
!ollama create pockitect:finetuned -f data/finetuning/output/Modelfile